In [121]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.text import Text
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.parsing.preprocessing import remove_stopwords
import pandas as pd
import re

In [122]:
stopwords = nltk.corpus.stopwords.words("english")
data = pd.read_csv('all_chains_cs_reviews.csv')
data['stars'] = data['stars_y']
data = data.drop(['Unnamed: 0', 'business_id', 'city', 'address', 'state', 'postal_code', 'latitude', 'longitude', 'attributes', 'categories', 'hours', 'review_id', 'useful', 'funny', 'cool', 'date', 'is_open', 'stars_x', 'stars_y', 'review_count'], axis=1)
data.head()

,name,user_id,text,stars
0,Popeyes,cZA_G7kIkyIrR15EKXoVFw,Awful. Waited in line for 15-20 for a chicken...,1.0
1,Popeyes,zKMCLxQnAOXpHJIKMZCI_Q,This is my first time at the Hartsfield Airpor...,1.0
2,Popeyes,CpRBM-El-mqvbv93lYX5QA,They weren't ready for the return of the chick...,4.0
3,Popeyes,ui1vL68Ty9_aeKGtzJNSHg,I have being several times there but they neve...,1.0
4,Popeyes,Lk_clm7vulcrkvcBAyO6fg,I have been trying to get the spicy chicken sa...,1.0


In [123]:
stars_1_2_reviews = pd.DataFrame(data[data['stars']==1.0])
stars_1_2_reviews = stars_1_2_reviews.append(data[data['stars']==2.0])
print("Number of 1 and 2 star chicken sandwich reviews =", len(stars_1_2_reviews))
stars_4_5_reviews = pd.DataFrame(data[data['stars']==4.0])
stars_4_5_reviews = stars_4_5_reviews.append(data[data['stars']==5.0])
print("Number of 4 and 5 star chicken sandwich reviews =", len(stars_4_5_reviews))

Number of 1 and 2 star chicken sandwich reviews = 1097
Number of 4 and 5 star chicken sandwich reviews = 1005


In [124]:
stars_1_2_text = ""
stars_4_5_text = ""
for i in range(len(stars_1_2_reviews)):
    stars_1_2_text = stars_1_2_text + stars_1_2_reviews["text"].iloc[i]

for j in range(len(stars_4_5_reviews)):
    stars_4_5_text = stars_4_5_text + stars_4_5_reviews["text"].iloc[j]

stars_1_2_text = re.sub("[^A-Za-z]+", " ", stars_1_2_text)
stars_4_5_text = re.sub("[^A-Za-z]+", " ", stars_4_5_text)

stars_1_2_text = Text(word_tokenize(stars_1_2_text))
stars_4_5_text = Text(word_tokenize(stars_4_5_text))

In [125]:
stars_1_2_con_list = stars_1_2_text.concordance_list(["chicken", "sandwich"], lines=1000)
stars_4_5_con_list = stars_4_5_text.concordance_list(["chicken", "sandwich"], lines=1000)

In [126]:
#make all cases lowercase and add to string
stars_1_2_string = ""
stars_4_5_string = ""
for i in range(len(stars_1_2_con_list)):
    stars_1_2_con_list[i] = remove_stopwords(stars_1_2_con_list[i].line).lower()
    stars_1_2_string = stars_1_2_string + stars_1_2_con_list[i]

for i in range(len(stars_4_5_con_list)):
    stars_4_5_con_list[i] = remove_stopwords(stars_4_5_con_list[i].line).lower()
    stars_4_5_string = stars_4_5_string + stars_4_5_con_list[i]

stars_1_2_string = re.sub('chicken sandwich', ' ', stars_1_2_string)
stars_4_5_string = re.sub('chicken sandwich', ' ', stars_4_5_string)

In [143]:
words_1_2 = word_tokenize(stars_1_2_string)
fdist_1_2 = FreqDist(words_1_2)

words_4_5 = word_tokenize(stars_4_5_string)
fdist_4_5 = FreqDist(words_4_5)

In [144]:
df_fdist_1_2 = pd.DataFrame.from_dict(fdist_1_2, orient='index')
df_fdist_1_2.columns = ['Frequency']
df_fdist_1_2.index.name = 'Term'
df_fdist_1_2 = df_fdist_1_2.sort_values(by=['Frequency'], ascending=False)
df_fdist_1_2 = df_fdist_1_2[(df_fdist_1_2.Frequency != 1) & (df_fdist_1_2.Frequency != 2)]

df_fdist_4_5 = pd.DataFrame.from_dict(fdist_4_5, orient='index')
df_fdist_4_5.columns = ['Frequency']
df_fdist_4_5.index.name = 'Term'
df_fdist_4_5 = df_fdist_4_5.sort_values(by=['Frequency'], ascending=False)
df_fdist_4_5 = df_fdist_4_5[(df_fdist_4_5.Frequency != 1) & (df_fdist_4_5.Frequency != 2)]

In [145]:
df_fdist_1_2 = df_fdist_1_2.drop(["i", "the", "got", "s", "took", "w", "going", "and", "this", "bun", "ordered", "t", "order", "spicy", "grilled", "fries", "chicken", "like", "meal", "new", "crispy", "try", "food", "asked", "my", "it", "went", "they", "combo", "we", "home", "so", "know", "m", "king", "wife", "fried", "classic", "asiago", "not", "fil", "best", "she", "tried", "free", "burger", "pickles", "fresh", "sauce", "location", "lettuce", "instead", "wendy", "mayo", "buttermilk", "large", "regular", "ve", "received", "drive", "worst", "no", "came", "cheese", "when", "nuggets", "bacon", "wanted", "chick", "told", "a", "good", "tomato", "warm", "bit", "said", "today",  "original", "sandwich", "gave"])
df_fdist_1_2.head(20).to_csv('df_fdist_1_2.csv')

In [146]:
df_fdist_4_5 = df_fdist_4_5.drop(["i", "the", "got", "s", "popeye", "hands", "right", "thought", "number", "come", "thing", "tenders", "salad", "biscuit", "comes", "enjoy", "fantastic", "with", "location", "spot", "there", "lunch", "wait", "wendy", "wrong", "ordered", "t", "order", "spicy", "grilled", "good", "best", "fries", "fil", "a", "love", "waffle", "chicken", "food", "deluxe", "chick", "original", "fried", "it", "better", "awesome", "they", "sauce", "place", "combo", "drive", "not", "but", "wheat", "definitely", "ice", "infamous", "sweet", "bun", "add", "tea", "ll", "and", "lettuce", "craving", "lemonade", "jack", "oh", "that", "day", "menu", "tasty", "famous", "finally", "know", "way", "today", "came", "pepper", "burger", "amazing", "delicious", "eat", "their", "we", "so", "usually", "favorite", "time", "classic", "popeyes", "sandwich", "this", "like", "m", "pickles", "cheese", "great", "try", "meal", "my", "regular", "ve", "nuggets"])
df_fdist_4_5.head(20).to_csv('df_fdist_4_5.csv')